In [1]:
from pathlib import Path
import os
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoConfig, AutoModelForCausalLM


MODEL_DIR = r"C:/Users/HP/Phi-3.5-vision-instruct"  # source HF repo
OUT_DIR = Path(r"C:/Users/HP/phi_onnx")
OUT_DIR.mkdir(parents=True, exist_ok=True)


# Load config override to avoid flash-attn
config = AutoConfig.from_pretrained(MODEL_DIR, trust_remote_code=True)
config._attn_implementation = "eager"
config.attn_implementation = "eager"


print("Loading PyTorch model (CPU)...")
pt_model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR, config=config, trust_remote_code=True, device_map="cpu", torch_dtype=None
)





c:\Users\HP\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading PyTorch model (CPU)...


Loading checkpoint shards:  50%|█████     | 1/2 [30:11<30:10, 1810.55s/it]

: 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoConfig
from PIL import Image
import requests

# Path to your local clone
model_path = r"C:\Users\HP\Phi-3.5-vision-instruct"

print("Loading model...")
# First load and modify the config
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
config._attn_implementation = "eager"  # Disable flash attention

# Now load the model with the modified config
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    config=config,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)



c:\Users\HP\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...


Loading checkpoint shards: 100%|██████████| 2/2 [16:07<00:00, 483.79s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.
c:\Users\HP\miniconda3\Lib\site-packages\transformers\models\auto\image_processing_auto.py:640: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [26]:
import torch
from pathlib import Path
from transformers import AutoProcessor
from PIL import Image

model.config.use_cache = False
model.eval()

out_dir = Path(r"C:\Users\HP\phi_onnx")
out_dir.mkdir(parents=True, exist_ok=True)

dummy_image = Image.new("RGB", (224, 224))
inputs = processor(text="Here is an image: <|image_1|>", images=[dummy_image], return_tensors="pt")
input_ids = inputs["input_ids"]
pixel_values = inputs["pixel_values"]

# Disable Dynamo globally
torch._dynamo.config.suppress_errors = True

onnx_path = out_dir / "phi3_vision.onnx"

torch.onnx.export(
    model,
    (input_ids, pixel_values),
    onnx_path.as_posix(),
    opset_version=14,
    export_params=True,
    do_constant_folding=True,
    input_names=["input_ids", "pixel_values"],
    output_names=["logits"],
    dynamic_axes={"input_ids": {0: "batch", 1: "seq"}, "pixel_values": {0: "batch"}, "logits": {0: "batch", 1: "seq"}}
)

processor.save_pretrained(out_dir)
print("ONNX exported (Dynamo errors suppressed):", onnx_path)

C:\Users\HP\AppData\Local\Temp\ipykernel_6612\1209062572.py:22: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1104 16:19:15.508000 6612 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 14 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`...


W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500] could not determine __code__ for Phi3VForCausalLM(
W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500]   (model): Phi3VModel(
W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_dropout): Dropout(p=0.0, inplace=False)
W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (vision_embed_tokens): Phi3ImageEmbedding(
W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (drop): Dropout(p=0.0, inplace=False)
W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (wte): Embedding(32064, 3072, padding_idx=32000)
W1104 16:19:26.986000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (img_processor): CLIPVisionModel(
W1104 16:19:26.986000 6612 site-pac

[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`... ❌
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`...
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`... ❌


TorchExportError: Failed to export the model with torch.export. [96mThis is step 1/3[0m of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and submit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the [96m*torch.export*[0m component and attach the full error stack as well as reproduction scripts.

## Exception summary

<class 'ValueError'>: could not determine the shape of object type 'torch.storage.UntypedStorage'

(Refer to the full stack trace above for more information.)

In [24]:
from pathlib import Path
import torch
from transformers import AutoProcessor
from PIL import Image

# Keep current env/model
model.config.use_cache = False
model.eval()

out_dir = Path(r"C:\Users\HP\phi_onnx")
out_dir.mkdir(parents=True, exist_ok=True)

dummy_image = Image.new("RGB", (224, 224))
inputs = processor(text="Here is an image: <|image_1|>", images=[dummy_image], return_tensors="pt")
input_ids = inputs["input_ids"]
pixel_values = inputs["pixel_values"]

onnx_path = out_dir / "phi3_vision.onnx"

# Force legacy exporter
torch.onnx.export(
    model,
    (input_ids, pixel_values),
    onnx_path.as_posix(),
    opset_version=14,
    do_constant_folding=True,
    export_params=True,
    input_names=["input_ids", "pixel_values"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch", 1: "seq"},
        "pixel_values": {0: "batch"},
        "logits": {0: "batch", 1: "seq"}
    },
    verbose=False
)

processor.save_pretrained(out_dir)
print("ONNX exported (opset 14):", onnx_path)

C:\Users\HP\AppData\Local\Temp\ipykernel_6612\1007985919.py:21: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1104 16:10:37.203000 6612 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 14 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W1104 16:10:48.099000 6612 site-packages\torch\_dynamo\eval_frame.py:500] could not determine __code__ for Phi3VForCausalLM(
W1104 16:10:48.099000 6612 site-packages\torch\_dynamo\eval

TorchExportError: Failed to export the model with torch.export. [96mThis is step 1/3[0m of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and submit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the [96m*torch.export*[0m component and attach the full error stack as well as reproduction scripts.

## Exception summary

<class 'ValueError'>: could not determine the shape of object type 'torch.storage.UntypedStorage'

(Refer to the full stack trace above for more information.)

In [23]:
from PIL import Image
from pathlib import Path
from transformers import AutoProcessor
import torch

if hasattr(model.config, "use_cache"):
    model.config.use_cache = False
model.eval()

out_dir = Path(r"C:\Users\HP\phi_onnx")
out_dir.mkdir(parents=True, exist_ok=True)

dummy_image = Image.new("RGB", (224, 224))
inputs = processor(text="Here is an image: <|image_1|>", images=[dummy_image], return_tensors="pt")
input_ids = inputs["input_ids"]
pixel_values = inputs["pixel_values"]

onnx_path = out_dir / "phi3_vision.onnx"

torch.onnx.export(
    model,
    (input_ids, pixel_values),
    onnx_path.as_posix(),
    opset_version=16,
    do_constant_folding=True,
    export_params=True,
    input_names=["input_ids", "pixel_values"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch", 1: "seq"},
        "pixel_values": {0: "batch"},
        "logits": {0: "batch", 1: "seq"}
    }
)

processor.save_pretrained(out_dir)
print("ONNX export done (opset 16):", onnx_path)

C:\Users\HP\AppData\Local\Temp\ipykernel_6612\1958307310.py:20: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W1104 15:56:50.750000 6612 site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 16 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`...


W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500] could not determine __code__ for Phi3VForCausalLM(
W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500]   (model): Phi3VModel(
W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_dropout): Dropout(p=0.0, inplace=False)
W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (vision_embed_tokens): Phi3ImageEmbedding(
W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (drop): Dropout(p=0.0, inplace=False)
W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (wte): Embedding(32064, 3072, padding_idx=32000)
W1104 15:57:00.572000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (img_processor): CLIPVisionModel(
W1104 15:57:00.572000 6612 site-pac

[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`... ❌
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`...
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`... ❌


TorchExportError: Failed to export the model with torch.export. [96mThis is step 1/3[0m of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and submit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the [96m*torch.export*[0m component and attach the full error stack as well as reproduction scripts.

## Exception summary

<class 'ValueError'>: could not determine the shape of object type 'torch.storage.UntypedStorage'

(Refer to the full stack trace above for more information.)

In [27]:
from PIL import Image
import torch
from pathlib import Path
from transformers import AutoProcessor

# Assume model is already in memory
# model = AutoModelForCausalLM.from_pretrained(...)

# Disable caching for export
if hasattr(model.config, "use_cache"):
    model.config.use_cache = False
model.eval()

# Output directory
out_dir = Path(r"C:\Users\HP\phi_onnx")
out_dir.mkdir(parents=True, exist_ok=True)

# Dummy inputs
batch_size = 1
seq_length = 8
image_size = 224

dummy_image = Image.new("RGB", (224, 224))  # single dummy image

inputs = processor(
    text="Here is an image: <|image_1|>",
    images=[dummy_image],  # must be a list
    return_tensors="pt"
)

dummy_input_ids = inputs["input_ids"]
dummy_pixel_values = inputs["pixel_values"]

# ONNX path
onnx_path = out_dir / "phi3_vision.onnx"

print("Exporting ONNX model from in-memory PyTorch model...")
torch.onnx.export(
    model,
    (dummy_input_ids, dummy_pixel_values),
    onnx_path.as_posix(),
    input_names=["input_ids", "pixel_values"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_length"},
        "pixel_values": {0: "batch_size", 2: "height", 3: "width"},
        "logits": {0: "batch_size", 1: "seq_length"}
    },
    opset_version=18,   # use latest
    do_constant_folding=True
)

print("Saving processor...")
processor.save_pretrained(out_dir)

print("\n✅ ONNX Export Completed Successfully!")
print(f"📁 Files saved to: {out_dir}")


Exporting ONNX model from in-memory PyTorch model...


C:\Users\HP\AppData\Local\Temp\ipykernel_6612\3044188919.py:38: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(


[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`...


W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500] could not determine __code__ for Phi3VForCausalLM(
W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500]   (model): Phi3VModel(
W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_dropout): Dropout(p=0.0, inplace=False)
W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (vision_embed_tokens): Phi3ImageEmbedding(
W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (drop): Dropout(p=0.0, inplace=False)
W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (wte): Embedding(32064, 3072, padding_idx=32000)
W1104 16:21:38.885000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (img_processor): CLIPVisionModel(
W1104 16:21:38.885000 6612 site-pac

[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`... ❌
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`...
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`... ❌


TorchExportError: Failed to export the model with torch.export. [96mThis is step 1/3[0m of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and submit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the [96m*torch.export*[0m component and attach the full error stack as well as reproduction scripts.

## Exception summary

<class 'ValueError'>: could not determine the shape of object type 'torch.storage.UntypedStorage'

(Refer to the full stack trace above for more information.)

In [19]:
import torch
from pathlib import Path
from transformers import AutoProcessor
from PIL import Image

# =============================
# 1️⃣ Load your model & processor
# =============================
# Example: adjust to your actual model path or checkpoint
# from transformers import AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained("path_or_model_name")
# processor = AutoProcessor.from_pretrained("path_or_model_name")

# For this snippet, we assume model and processor are already loaded in memory
# model = ...
# processor = ...

# =============================
# 2️⃣ Configure model for ONNX export
# =============================
if hasattr(model.config, "use_cache"):
    model.config.use_cache = False   # disable past_key_values caching
model.eval()                        # set to evaluation mode

# =============================
# 3️⃣ Prepare dummy inputs
# =============================
batch_size = 1
seq_length = 8
image_size = 224  # adjust if your model expects a different size

# Create a dummy image
dummy_image = Image.new("RGB", (image_size, image_size))

# Processor requires:
# - text string containing <|image_1|> tag for each image
# - images as a list
inputs = processor(
    text="Here is an image: <|image_1|>",
    images=[dummy_image],
    return_tensors="pt"
)

dummy_input_ids = inputs["input_ids"]
dummy_pixel_values = inputs["pixel_values"]

# =============================
# 4️⃣ Define output directory and ONNX path
# =============================
out_dir = Path(r"C:\Users\HP\phi_onnx")
out_dir.mkdir(parents=True, exist_ok=True)
onnx_path = out_dir / "phi3_vision.onnx"

# =============================
# 5️⃣ Export to ONNX
# =============================
torch.onnx.export(
    model,
    (dummy_input_ids, dummy_pixel_values),
    onnx_path.as_posix(),
    input_names=["input_ids", "pixel_values"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_length"},
        "pixel_values": {0: "batch_size", 2: "height", 3: "width"},
        "logits": {0: "batch_size", 1: "seq_length"}
    },
    opset_version=18,
    do_constant_folding=True
)

# =============================
# 6️⃣ Save processor for later use
# =============================
processor.save_pretrained(out_dir)

# =============================
# 7️⃣ Completion message
# =============================
print("\n✅ ONNX Export Completed Successfully!")
print(f"📁 Files saved to: {out_dir}")


C:\Users\HP\AppData\Local\Temp\ipykernel_6612\2249568028.py:57: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(


[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`...


W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500] could not determine __code__ for Phi3VForCausalLM(
W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500]   (model): Phi3VModel(
W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (embed_dropout): Dropout(p=0.0, inplace=False)
W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500]     (vision_embed_tokens): Phi3ImageEmbedding(
W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (drop): Dropout(p=0.0, inplace=False)
W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (wte): Embedding(32064, 3072, padding_idx=32000)
W1104 15:49:56.774000 6612 site-packages\torch\_dynamo\eval_frame.py:500]       (img_processor): CLIPVisionModel(
W1104 15:49:56.774000 6612 site-pac

[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=False)`... ❌
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`...
[torch.onnx] Obtain model graph for `Phi3VForCausalLM([...]` with `torch.export.export(..., strict=True)`... ❌


TorchExportError: Failed to export the model with torch.export. [96mThis is step 1/3[0m of exporting the model to ONNX. Next steps:
- Modify the model code for `torch.export.export` to succeed. Refer to https://pytorch.org/docs/stable/generated/exportdb/index.html for more information.
- Debug `torch.export.export` and submit a PR to PyTorch.
- Create an issue in the PyTorch GitHub repository against the [96m*torch.export*[0m component and attach the full error stack as well as reproduction scripts.

## Exception summary

<class 'ValueError'>: could not determine the shape of object type 'torch.storage.UntypedStorage'

(Refer to the full stack trace above for more information.)

In [ ]:
import torch
from pathlib import Path
from transformers import AutoProcessor

# Ensure your model is in memory
# model = AutoModelForCausalLM.from_pretrained(...)

# Disable caching for ONNX export
if hasattr(model.config, "use_cache"):
    model.config.use_cache = False

# Set model to evaluation mode
model.eval()

# Output directory
out_dir = Path(r"C:\Users\HP\phi_onnx")
out_dir.mkdir(parents=True, exist_ok=True)

# Dummy inputs (minimal size for tracing)
batch_size = 1
seq_length = 8
image_size = 224

dummy_input_ids = torch.randint(
    0, model.config.vocab_size, (batch_size, seq_length), dtype=torch.long
)
dummy_pixel_values = torch.zeros(batch_size, 3, image_size, image_size, dtype=torch.float32)

onnx_path = out_dir / "phi3_vision.onnx"

print("Exporting ONNX model from in-memory PyTorch model...")

torch.onnx.export(
    model,
    (dummy_input_ids, dummy_pixel_values),
    onnx_path.as_posix(),
    input_names=["input_ids", "pixel_values"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "seq_length"},
        "pixel_values": {0: "batch_size", 2: "height", 3: "width"},
        "logits": {0: "batch_size", 1: "seq_length"}
    },
    opset_version=18,        # Use 18 to match PyTorch 2.x
    do_constant_folding=True,
    strict=False             # Relax strict mode to avoid Dynamo tracing issues
)

print("Saving processor...")
processor.save_pretrained(out_dir)

print("\n✅ ONNX Export Completed Successfully!")
print(f"📁 Files saved to: {out_dir}")


In [ ]:
from pathlib import Path
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoProcessor, AutoConfig

model_path = r"C:/Users/HP/Phi-3.5-vision-instruct"
OUT_DIR = Path(r"C:/Users/HP/phi_onnx")
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("Loading config & disabling FlashAttention2...")
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
config._attn_implementation = "eager"  # ✅ Force CPU attention
config.flash_attn_2_enabled = False    # ✅ Disable FA2
config._flash_attn_2_enabled = False   # ✅ Some models use internal version

print("Saving patched config...")
config.save_pretrained(model_path)

print("Loading processor...")
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

print("Exporting ONNX (CPU Only)...")
onnx_model = ORTModelForCausalLM.from_pretrained(
    model_id=model_path,
    export=True,
    trust_remote_code=True,
    provider="CPUExecutionProvider",
    file_name="phi3_vision.onnx",
    use_external_format=True,
)

print("Saving ONNX + Processor...")
onnx_model.save_pretrained(OUT_DIR)
processor.save_pretrained(OUT_DIR)

print("\n✅ ONNX Export Completed Successfully!")
print(f"📁 Files saved to: {OUT_DIR}")


Loading config & disabling FlashAttention2...
Saving patched config...
Loading processor...


`file_name` was set to `phi3_vision.onnx` but will be ignored as the model will be converted to ONNX


Exporting ONNX (CPU Only)...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print("Exporting to ONNX with Optimum (this can take a while)...")
# ORTModelForCausalLM.from_pretrained can accept a PyTorch model object OR a path in recent optimum versions.
# We try direct export using the model object and saving into OUT_DIR
ORTModelForCausalLM.from_pretrained(
    pt_model,
    save_dir=str(OUT_DIR),
    file_name="phi3_vision.onnx",
)


print("Export complete. ONNX saved to:", OUT_DIR)